## Yolo V5 in Azure ML Environment

### Use the Python 3.8 Pytorch & Tensor Flow Environment

In [1]:
!ls

test  train  valid  yolo.ipynb


In [2]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

Cloning into 'yolov5'...
remote: Enumerating objects: 12960, done.
remote: Total 12960 (delta 0), reused 0 (delta 0), pack-reused 12960
Receiving objects: 100% (12960/12960), 11.88 MiB | 12.85 MiB/s, done.
Resolving deltas: 100% (9002/9002), done.
Updating files: 100% (108/108), done.
/mnt/batch/tasks/shared/LS_root/mounts/clusters/mmgpu02/code/Users/memasanz/yolov5_with_vector/yolov5
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch
from yolov5 import utils
display = utils.notebook_init()  # checks

In [ ]:
import torch
from IPython.display import Image  # for displaying images

print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
!ls

In [ ]:
# !curl -L "https://public.roboflow.com/ds/pDNZZrmMRc?key=rPjNXNeyPh" -o roboflow.zip; unzip roboflow.zip -d "./yolov5/";

In [ ]:
%cd ./yolov5

In [ ]:
%%writefile ./data.yaml

train: train/images
val: valid/images

nc: 1
names: ['vector']

In [ ]:
%%writefile ./custom_yolov5s.yaml

# parameters
nc: 1  # number of classes  # CHANGED HERE
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

In [ ]:
!python train.py --img 640 --batch 16 --epochs 3 --data data.yaml --weights yolov5s.pt

In [ ]:
experiments = !ls './runs/train/'
print(experiments)
print(experiments[-1])

In [ ]:
print("GROUND TRUTH TRAINING DATA:")
from utils.plots import plot_results
from IPython.display import Image, clear_output  # to display images
Image(filename='./runs/train/' + experiments[-1] +  '/val_batch0_labels.jpg', width=900)
#!ls ./runs/train/exp7/val_batch0_labels.jpg

In [ ]:
import pandas as pd
df = pd.read_csv('./runs/train/' + experiments[-1] + '/results.csv')

In [ ]:
df

In [ ]:
from utils.plots import plot_results  # plot results.txt as results.png
Image(filename='./runs/train/' + experiments[-1] + '/results.png', width=1000)

In [ ]:
#!ls ./runs/train/'exp7'/weights
import os
for root, dirs, files in os.walk('./runs/train/' + experiments[-1] + '/weights', topdown=False):
    for name in files:
        print(name)